In [ ]:
# /work/vmo703/scratch/repro_fig9.py
from PositronIPOLEScripts.ipole_positron_test_unlocked import makePositronImages

sim   = "/work/vmo703/grmhd_dump_samples/Sa-0.5_5000.h5"
Munit = 1.83e27  # your base Munit for SANE a=-0.5 (from your table)
out   = "/work/vmo703/ipole_outputs/fig9_test.h5"
ipole = "/work/vmo703/aricarte-copy/aricarte/ipole+e-/ipole"

makePositronImages(sim, Munit,
                   positronRatios=[0,1],
                   nameBase=out, inclination=163.0, Rhigh=20,
                   ipole=ipole, fov=160.0, npixel=320, counterjet=0, rmax_geo=50)


- import csv data for correct M_base
- loop over grmhd_dump_samples/
- expand nameBase and simFile dynamically from filename
- automate output.par and .h5 saving

potentially separate the dump samples output in a Dict like below to allow a batch run on each setting? RBETA[wJET] && CRITBETA[wJET]

In [ ]:
def process_file(file_path: str, bolo_groups: Dict[str, List[str]]) -> Dict[Tuple[str, int], List[float]]:
    """
    this function processes a time constant data file and returns the processed results

    args:
        file_path: the path to the time constant data file (in pickle format)
        bolo_groups: a dictionary that groups bolometers by some criteria (e.g., frequency)
            keys are group names, values are lists of detector IDs belonging to that group

    returns:
        a dictionary containing the processed time constant data:
            - keys are tuples of (frequency, elevation)
                values are lists containing the median time constant for each detector within that frequency-elevation pair
        if the function fails to extract obs_id or elevation from the filename, an empty dictionary is returned
    """
    match = re.search(r'time_const_(\d+)_el_(\d+)deg.pkl', file_path)

    if match:
        obs_id = int(match.group(1))
        el = int(match.group(2))
    else:
        logging.error(f"Failed to extract obs_id or elevation from filename: {file_path}")
        return {}

    with open(file_path, 'rb') as file:
        time_constant_data = pickle.load(file)
        return process_time_constant_data(time_constant_data, bolo_groups, el, obs_id)